In [2]:
import torch
from examples.vae_mnist import VAE_MNIST
from loss import ELBO
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'examples'

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_dataset = datasets.MNIST('./_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

test_dataset = datasets.MNIST('./_mnist', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ]))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
def train(model, optim, dataloader, no_epochs=3):
    model.train()
    
    train_loss = []
    for epoch in range(no_epochs):
        for imgs, _ in tqdm(dataloader, desc="Training Epoch №%s" % epoch, leave=False):
            imgs = imgs.view(-1, 28*28)
            vae_res = model(imgs)
            loss = ELBO(vae_res)
            train_loss.append(loss.item())

            optim.zero_grad()
            loss.backward()
            optimizer.step()

    return torch.Tensor(train_loss)

In [ ]:
model = VAE_MNIST()
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [2]:
train(model, optim, train_loader)

NameError: name 'train' is not defined

In [6]:
help('modules')


Please wait a moment while I gather a list of all available modules...



NOTE: Redirects are currently not supported in Windows or MacOs.


IPython             array               ipaddress           runpy
OpenSSL             ast                 ipykernel           sched
PIL                 asttokens           ipykernel_launcher  secrets
PyQt5               asynchat            ipython_genutils    select
__future__          asyncio             ipywidgets          selectors
_abc                asyncore            itertools           send2trash
_aix_support        atexit              jedi                setuptools
_argon2_cffi_bindings attr                jinja2              shell_exec
_ast                attrs               json                shelve
_asyncio            audioop             jsonschema          shlex
_bisect             backcall            jupyter             shutil
_blake2             backports           jupyter_client      signal
_bootlocale         base64              jupyter_console     sip
_bootsubprocess     bdb                 jupyter_core        sipconfig
_bz2                binascii            jupyter

/Users/andy/opt/anaconda3/envs/elbo-comparison/lib/python3.9/pkgutil.py:108: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)
